# Instalando dependências

In [92]:
%pip install pandas
%pip install joblib
%pip install tiktoken
%pip install matplotlib
%pip install -U google-generativeai
%pip install python-decouple
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Carregando a base de dados dos simpsons

In [9]:
import pandas as pd

df_script = pd.read_csv('./data/thesimpsons/simpsons_script_lines.csv', low_memory=False)
df_episodes = pd.read_csv('./data/thesimpsons/simpsons_episodes.csv', low_memory=False)
df_characters = pd.read_csv('./data/thesimpsons/simpsons_characters.csv', low_memory=False)
df_locations = pd.read_csv('./data/thesimpsons/simpsons_locations.csv', low_memory=False)

df_script.set_index('id', inplace=True)
df_characters['id'] = df_characters['id'].astype(str)

df_characters = df_characters.add_prefix('character_')
df_locations = df_locations.add_prefix('location_')
df_episodes = df_episodes.add_prefix('episode_')

data = (
    df_script.merge(df_episodes, left_on='episode_id', right_on='episode_id')
             .merge(df_characters, left_on='character_id', right_on='character_id', how='left')
             .merge(df_locations, left_on='location_id', right_on='location_id', how='left')
)


assert data.shape[0] == df_script.shape[0]

In [ ]:
import joblib
import pandas as pd
from dotenv import load_dotenv

load_dotenv('../.env')


# Carga dos resumos
summaries = joblib.load('../data/results/simpsons_episode_summary.joblib')
print(summaries.keys())

data = pd.read_parquet('../data/results/database_thesimpsons.parquet')


data['line'] = ("Espisode " + data['episode_id'].astype(str) + ' | ' + 
                data['location_normalized_name'].fillna('') + ', ' + 
                data['character_normalized_name'].fillna('') + ' said: ' + 
                data['normalized_text'].fillna('')
)

print(data.columns)

# Estimativa de Tokens

In [14]:
import tiktoken

def get_tokens(text: str):
    encoder = tiktoken.get_encoding("cl100k_base")
    tokens = encoder.encode(text)
    return tokens


In [44]:
X = data.dropna(subset='normalized_text').copy()
X['n_tokens'] = X.normalized_text.fillna('').apply(lambda x: len(get_tokens(x)))
X.shape

(132087, 31)

# Calculando a média de tokens por episódio

In [47]:
token_per_episode = X.groupby('episode_id').n_tokens.sum()
general_token_mean = token_per_episode.mean()
most_tokens_episode = token_per_episode.idxmax()

print("O episódio com mais tokens é o", most_tokens_episode, "com", token_per_episode[most_tokens_episode], "tokens.")
print("A média de tokens por episódio é de {:.2f}".format(general_token_mean))

O episódio com mais tokens é o 49 com 3454 tokens.
A média de tokens por episódio é de 2606.98


# Calculando a média de tokens por temporada

In [51]:
token_per_season = X.groupby('episode_season').n_tokens.sum()
general_token_mean = token_per_season.mean()
most_tokens_season = token_per_season.idxmax()

print("A season com mais tokens é a", most_tokens_season, "com", token_per_season[most_tokens_season], "tokens.")
print("A média de tokens por temporada é de", general_token_mean)

A season com mais tokens é a 7 com 70418 tokens.
A média de tokens por temporada é de 56551.46153846154


# Prompt Chaining para fazer uma análise descritiva das avaliações do IMDB e da audiência dos episódios

In [81]:
cols = ['episode_id', 'episode_season', 'episode_imdb_rating', 'episode_views']
episode_stats = data[cols].drop_duplicates()
episode_stats.to_csv('episode_stats.csv', sep=';', index=None)

# Levantando dados importantes sobre o dataset

In [104]:
import google.generativeai as genai
from decouple import config
from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=config("GEMINI_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

csv_input = pd.read_csv('episode_stats.csv', nrows=100)

prompt = f"""
You are a data scientist specialized in creating descriptive analisys. 
['episode_id', 'episode_season', 'episode_imdb_rating', 'episode_views'] in series_data.csv.

- series_data.csv: is a CSV file splitted by ';' and with the following columns:

- episode_id: unique episode index
- episode_season: episode season number
- episode_imdb_rating: IMDB rating of the episode 
- episode_views: total views of the episode in milions.

- All the data is from THE SIMPSONS series.

1. The goal is to try to data, trends and a meaningful numbers of the data. 
3. make a mean of views and rating per season.
4. Bring relevant data.
5. do not add any thoughts or opinions.
6. Talk only about data
7. Do not Talk abou code or programming. Just data.
8. Give the max quanity of data possible.


<series_data.csv>
    {csv_input}
</series_data.csv>
"""

response = model.generate_content(prompt)

important_data = response.text

# Divagando sobre os dados

In [105]:
prompt = f"""
You are a data scientist specialized in creating insights about data

You will recive a bunch of numbers, insights and trends about a dataset.

1. The goal is to try to capture insights, trends and a meaningful analysis of the data. 
2. Just talk about the data, don't worry about the code.
3. Talk A Lot of the data
4. Make all the analysis you can.
5. add thoughts and opinions.
6. Allways be very detailed and specific.


<data>
    {important_data}
</data>
"""

important_insights = model.generate_content(prompt).text

# Compilando em uma resposta final

In [113]:
prompt = f"""
You are a data scientist specialized in Teaching and resuming important Data

You will recive a bunch of insights and information about a dataset.

1. The goal is resume the data in a very detailed way.
2. Just talk about the data, don't worry about the code.
3. Resume the data in a very detailed way.
4. Make everything very clear.
5. add thoughts and opinions.
6. Make a very detailed, but small resume of the data
7. Be concise and clear. but do not miss any important information.

<data>
    {important_data}
</data>
"""

important_insights = model.generate_content(prompt).text

In [114]:
important_insights

"This dataset presents a descriptive analysis of IMDB ratings and viewership (in millions) for a sample of 100 Simpsons episodes drawn from seasons 2, 6, and 12.  The data is incomplete, representing only a fraction of the entire Simpsons series.  Therefore, generalizations about the entire show's performance should be avoided.\n\n**Key Findings:**\n\n* **Season-to-Season Variability:**  Significant differences exist in both average IMDB ratings and viewership across the three sampled seasons. Season 6 boasts the highest average IMDB rating (8.4) and viewership (69.96 million), while Season 12 shows the lowest average IMDB rating (7.15) and relatively lower viewership (49.44 million). Season 2 falls between the two.  This suggests a potential decline in both critical reception and viewership as the show progressed (at least within the sampled seasons).  This trend, however, might not hold for the whole series.\n\n* **Within-Season Variability:** The dataset indicates considerable varia